In [3]:
import pandas as pd
import numpy as np
import torch
import torchtext
import random
from torch import nn
from sklearn.metrics import f1_score
from nltk import word_tokenize
from torch import optim

random_state = random.getstate()
batch_size = 64

text = torchtext.data.Field(lower=True, batch_first=True, tokenize=word_tokenize)
qid = torchtext.data.Field()
target = torchtext.data.Field(sequential=False, use_vocab=False, is_target=True)
train = torchtext.data.TabularDataset(path='../input/train.csv', format='csv',
                                      fields={'question_text': ('text',text),
                                              'target': ('target',target)})


text.build_vocab(train, min_freq=1)
text.vocab.load_vectors(torchtext.vocab.Vectors('../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'))
print(text.vocab.vectors.shape)


train, val = train.split(split_ratio=0.8, random_state=random_state)


train_iter = torchtext.data.BucketIterator(dataset=train,
                                           batch_size=batch_size,
                                           sort_key=lambda x: x.text.__len__(),
                                           shuffle=True,
                                           sort=False)

val_iter = torchtext.data.BucketIterator(dataset=val,
                                         batch_size=batch_size,
                                         sort_key=lambda x: x.text.__len__(),
                                         train=False,
                                         sort=False)

In [ ]:
def training(epoch, model, loss_func, optimizer, train_iter, val_iter):
    step = 0
    train_record = []
    losses = []
    
    for e in range(epoch):
        train_iter.init_epoch()
        for train_batch in iter(train_iter):
            step += 1
            model.train()
            x = train_batch.text.cuda()
            y = train_batch.target.type(torch.Tensor).cuda()
            model.zero_grad()
            pred = model.forward(x).view(-1)
            loss = loss_function(pred, y)
            loss_data = loss.cpu().data.numpy()
            train_record.append(loss_data)
            loss.backward()
            optimizer.step()
            if step % 1000 == 0:
                print("Step: {:06}, loss {:.4f}".format(step, loss_data))
            if step % 10000 == 0:
                model.eval()
                model.zero_grad()
                val_loss = []
                for val_batch in iter(val_iter):
                    val_x = val_batch.text.cuda()
                    val_y = val_batch.target.type(torch.Tensor).cuda()
                    val_pred = model.forward(val_x).view(-1)
                    val_loss.append(loss_function(val_pred, val_y).cpu().data.numpy())
                val_record.append({'step': step, 'loss': np.mean(val_loss)})
                print('Epoch {:02} - step {:06} - train_loss {:.4f} - val_loss {:.4f} '.format(
                            e, step, np.mean(train_record), val_record[-1]['loss']))
                train_record = []

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, pretrained_lm, padding_idx, hidden_dim, static=True):
        super(SimpleModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding.from_pretrained(pretrained_lm)
        self.embedding.padding_idx = padding_idx
        if static:
            self.embedding.weight.requires_grad = False
        # TODO
    
    def forward(self, sents):
        x = self.embedding(sents)
        # TODO 
        return output

In [ ]:
model = SimpleModel(text.vocab.vectors,
                    padding_idx=text.vocab.stoi[text.pad_token],
                    hidden_dim=128).cuda()
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=1e-3)

training(model=model,
         epoch=20,
         loss_func=loss_function,
         optimizer=optimizer,
         train_iter=train_iter,
         val_iter=val_iter)
